In [8]:
import pandas as pd
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

ModuleNotFoundError: No module named 'sklearn'

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
train = train[['id', 'excerpt', 'target']]

In [3]:
def preprocessing(df):
    function = lambda x: x.count(',')
    df['comma'] = df['excerpt'].apply(function)
    
    function = lambda x: x.count('.')
    df['dot'] = df['excerpt'].apply(function)
    
    function = lambda x: x.count(' ')
    df['space'] = df['excerpt'].apply(function)
    
    function = lambda x: x.split()
    df['words'] = df['excerpt'].apply(function)
    function = lambda x: len(x)
    df['words_length'] = df['words'].apply(function)
    
    function = lambda x: list(x)
    df['letters'] = df['excerpt'].apply(function)
    function = lambda x: len(x)
    df['letters_length'] = df['letters'].apply(function)
    
    df.drop(columns=['words','letters'], inplace=True)


In [4]:
preprocessing(train)
preprocessing(test)

In [6]:
train.head()

,id,excerpt,target,comma,dot,space,words_length,letters_length
0,c12129c31,When the young people returned to the ballroom...,-0.340259,14,11,173,179,992
1,85aa80a4c,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,24,10,163,169,937
2,b69ac6792,"As Roger had predicted, the snow departed as q...",-0.580118,17,11,161,166,908
3,dd1000b26,And outside before the palace a great garden w...,-1.054013,23,5,162,164,909
4,37c1b32fb,Once upon a time there were Three Bears who li...,0.247197,13,5,146,147,723


In [7]:
categorical_feature=['space','dot','comma','words_length','letters_length']
Id = test['id']
train = train[['space','dot','comma','words_length','letters_length','target']]
test = test[categorical_feature]

from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(train, train['target'],
                                                      test_size=0.3, random_state=0)

X_train.drop(columns='target', inplace=True)
X_valid.drop(columns='target', inplace=True)

from sklearn.model_selection import StratifiedKFold

# 5-fold CVモデルの学習
# 5つのモデルを保存するリストの初期化
models = []
pred_ave = []
first_judge = True
num_fold = 6

# 学習データの数だけの数列（0行から最終行まで連番）
row_no_list = list(range(len(y_train)))

# KFoldクラスをインスタンス化（これを使って5分割する）
K_fold = StratifiedKFold(n_splits=num_fold, shuffle=True, random_state=42)

# KFoldクラスで分割した回数だけ実行（ここでは5回）
for train_cv_no, eval_cv_no in K_fold.split(row_no_list, y_train):
    # ilocで取り出す行を指定
    X_train_cv = X_train.iloc[train_cv_no, :]
    y_train_cv = pd.Series(y_train).iloc[train_cv_no]
    X_eval_cv = X_train.iloc[eval_cv_no, :]
    y_eval_cv = pd.Series(y_train).iloc[eval_cv_no]

    # 学習用
    lgb_train = lgb.Dataset(X_train_cv, y_train_cv,
                            categorical_feature=categorical_feature)
    # 検証用
    lgb_eval = lgb.Dataset(X_eval_cv, y_eval_cv, reference=lgb_train,
                           categorical_feature=categorical_feature)

    # パラメータを設定
    params = {'objective': 'regression',
              'metric': 'mse',
              #               'learning_rate':0.1,
              #               'num_iterations':100,
              #               'num_leaves':31,
              #               'max_depth':-1
              #               'weight_columns':[0.07495,0.110919,0.065571,0.067531,0.05158,
              #                                0.10085,0.053923,0.147468,0.065578,0.060578,
              #                                0.10546,0.095594]
              }

    # 学習
    evaluation_results = {}  # 学習の経過を保存する箱
    model = lgb.train(params,  # 上記で設定したパラメータ
                      lgb_train,  # 使用するデータセット
                      num_boost_round=1000,  # 学習の回数
                      valid_sets=[lgb_train, lgb_eval],  # モデル検証のデータセット
                      categorical_feature=categorical_feature,  # カテゴリー変数を設定
                      early_stopping_rounds=100,  # アーリーストッピング# 学習
                      verbose_eval=10)  # 学習の経過の非表示

    # テストデータで予測する
    y_pred = model.predict(test, num_iteration=model.best_iteration)

    if first_judge:
        pred_ave = y_pred
        first_judge = False
    else:
        pred_ave = pred_ave + y_pred

    # 学習が終わったモデルをリストに入れておく
    models.append(model)

pred_ave = pred_ave / num_fold

first_judge = True

for model in models:
    x_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    if first_judge:
        x_pred_ave = x_pred
        first_judge = False
    else:
        x_pred_ave = x_pred_ave + x_pred

x_pred_ave = x_pred_ave / num_fold
np.sqrt(mean_squared_error(y_valid, x_pred_ave))

# my_solution = pd.DataFrame(pred_ave, Id, columns=['SalePrice'])
# my_solution.to_csv("my_prediction_data.csv", header=False)

ModuleNotFoundError: No module named 'sklearn'